# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [1]:
import os
from dotenv import load_dotenv
#from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
api_version = os.getenv('OPENAI_API_VERSION')
azure_endpoint = os.getenv('OPENAI_ENDPOINT')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
elif not api_version:
    print("No API version found - please add OPENAI_API_VERSION to your .env file")
elif not azure_endpoint:
    print("No Azure endpoint found - please add OPENAI_ENDPOINT to your .env file")
else:
    print("Azure OpenAI credentials found and look good!")

Azure OpenAI credentials found and look good!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [2]:
import requests

# Azure OpenAI uses different headers - api-key instead of Authorization Bearer
headers = {"api-key": api_key, "Content-Type": "application/json"}

payload = {
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [3]:
# Use Azure OpenAI endpoint - format: {azure_endpoint}/openai/deployments/{model}/chat/completions?api-version={api_version}
response = requests.post(
    f"{azure_endpoint}/openai/deployments/gpt-4o/chat/completions?api-version={api_version}",
    headers=headers,
    json=payload
)

response.json()

{'choices': [{'content_filter_results': {'hate': {'filtered': False,
     'severity': 'safe'},
    'protected_material_code': {'filtered': False, 'detected': False},
    'protected_material_text': {'filtered': False, 'detected': False},
    'self_harm': {'filtered': False, 'severity': 'safe'},
    'sexual': {'filtered': False, 'severity': 'safe'},
    'violence': {'filtered': False, 'severity': 'safe'}},
   'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'annotations': [],
    'content': "Sure! Here's a fun fact: Octopuses have three hearts and their blood is blue! Two of their hearts pump blood to their gills, while the third pumps it to the rest of their body. The blue color comes from a copper-based molecule called hemocyanin, which is better suited for transporting oxygen in cold, low-oxygen environments where many octopuses live. Nature is pretty cool, huh?",
    'refusal': None,
    'role': 'assistant'}}],
 'created': 1772218215,
 'id': 'chatcmpl-DDxCh

In [4]:
response.json()["choices"][0]["message"]["content"]

"Sure! Here's a fun fact: Octopuses have three hearts and their blood is blue! Two of their hearts pump blood to their gills, while the third pumps it to the rest of their body. The blue color comes from a copper-based molecule called hemocyanin, which is better suited for transporting oxygen in cold, low-oxygen environments where many octopuses live. Nature is pretty cool, huh?"

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [5]:
# Create Azure OpenAI client

from openai import AzureOpenAI

azure_openai = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint
)

response = azure_openai.chat.completions.create(
    model="gpt-4o",  # Your Azure deployment name
    messages=[{"role": "user", "content": "Tell me a fun fact"}]
)

response.choices[0].message.content



'Sure! Here\'s a fun fact: A group of flamingos is called a "flamboyance." How fitting is that for such brightly colored, elegant birds?'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

## THIS IS OPTIONAL - but if you wish to try out Google Gemini, please visit:

https://aistudio.google.com/

And set up your API key at

https://aistudio.google.com/api-keys

And then add your key to the `.env` file, being sure to Save the .env file after you change it:

`GOOGLE_API_KEY=AIz...`


In [6]:
from openai import OpenAI

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini


In [7]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-flash-lite", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}]

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [ ]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [8]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 6.7 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   2% ▕                  ▏  44 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   3% ▕                  ▏  66 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   5% ▕                  ▏ 105 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   7% ▕█                 ▏ 134 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   7% ▕█                 ▏ 148 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   8% ▕█                 ▏ 152 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  10% ▕█                 ▏ 203 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  10% ▕█                 ▏ 203 MB/2.0 GB            

In [9]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [10]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Here\'s a fun fact:\n\nDid you know that there is a species of jellyfish that is immortal? The Turritopsis dohrnii, also known as the "immortal jellyfish," can transform its body into a younger state through a process called transdifferentiation. This means it can essentially revert back to its polyp stage and grow back into an adult again, making it theoretically immortal!'

In [11]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  17 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   6% ▕█                 ▏  65 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   8% ▕█                 ▏  91 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  13% ▕██                ▏ 147 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  17% ▕███               ▏ 188 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  19% ▕███               ▏ 210 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  23% ▕████              ▏ 254 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  26% ▕████              ▏ 295 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:  29% ▕█████             ▏ 323 MB/1.1 GB                  pulling manifest 
pulling aabd4d

In [12]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Sure! Here's a fun fact: **A slice of bread is nearly a whole being, though it has no internal structure**."

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [15]:
from scraper import fetch_website_contents
from IPython.display import Markdown, display

In [16]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [17]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

In [18]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [19]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = ollama.chat.completions.create(
        model = "deepseek-r1:1.5b",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
summarize("https://edwarddonner.com")

'The website features a home page titled "Home," authored by Edward Donner, and highlights several AI-related content, including blog posts and AI courses from his site. There\'s no mention of any nav links beyond the site itself.\n\n**Summary:**  \nEdward Donner author homepage with references to his AI programs and blog posts.  \n\nIf you\'re looking for info on LLMs or AI courses via "Nebula.io" or "unc attained," let me know if it pokes your interest!  \n\n```'

In [21]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [22]:
display_summary("https://edwarddonner.com")

It's Edward Donner's AI and Neural Networks corner of the internet, powered by the latest AAAI projects. Check out Vibe Coder in January, the ultimate AI engineering course soon from Ed, and stay tuned to the live energy of AI events.